In [13]:
!pip install openai==1.25.1

In [14]:
!pip install httpx==0.27.2

In [28]:
import os
import requests
from bs4 import BeautifulSoup #ウェブスクレイピングでHTMLデータを解析し、データ抽出
from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata

os.environ["OPEN_API_KEY_PROD"] = userdata.get('OPEN_API_KEY_PROD')
# OpenAIクライアントを初期化
client = OpenAI(api_key= os.environ["OPEN_API_KEY_PROD"])

# 先ほど作成したスクレイピング処理の関数化
def scrape_article(url):
# 対象のURLを取得（スクレイピング処理）
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser") #対象urlのhtmlを取得
  print(soup)

  # テキストの情報だけを取得してくる
  text_nodes = soup.find_all("div")
  len(text_nodes) #divで囲まれてる箇所何か所あるか確認
  for t in text_nodes:
    print(t.text) #t.textでtext属性のものだけを取得できる

  # 上記テキスト情報をリストへ格納
  t_all = []
  for t in text_nodes:
    t_all.append(t.text.replace("\n","").replace("\t","")) #\n,\tを""に変換"
    print(t_all)

  #文字列をまずはすべて結合させる
  joined_text =  "".join(t_all)
  return joined_text


# chunkの取得処理も関数化
# 400文字で一区切り、50文字戻り350+50で区切っていく
def chunk_text(text,chunk_size,overlap,):
  chunks = []
  start = 0

  while start + chunk_size <= len(text): #この文字数までは回し続ける
    chunks.append(text[start:start + chunk_size]) #最初は0:400文字まではいってる
    start += (chunk_size - overlap)
  #上記処理では最後が例えばstart:1100だと残り分を残したまま処理が終わってしまうため残り分を算出
  if start < len(text):
    chunks.append(text[-chunk_size:])

  return chunks

# テキストをベクトル化する関数処理流用
  #client.embeddingsでembeddingsAPIをcallする
def vectorize_text(text):
  response = client.embeddings.create(
      input = text,
      model = "text-embedding-3-small"
  )
  return response.data[0].embedding

# documentsの中で最も高い類似度を格納する処理も関数化
def find_most_similar(question_vector, vectors, documents):
  max_similarity = 0
  most_similar_index = 0

  for index, vector in enumerate(vectors):
    similarity = cosine_similarity([question_vector], [vector])[0][0]
    print(documents[index], ":", similarity) #処理の流れ確認
    if similarity > max_similarity:
      max_similarity = similarity
      most_similar_index = index

  return documents[most_similar_index]

# 質問への回答を出力するプロンプトも関数化
def ask_question(question_context, context):
  prompt = f''' 以下の質問に以下情報から答えてください。
  [ユーザへの質問]
  {question}

  [情報]
  {context}
  '''
  print(prompt)
  response = client.completions.create(
      model = "gpt-3.5-turbo-instruct",
      prompt=prompt,
      max_tokens = 200
  )

  return response.choices[0].text


### 実際にWEBページの情報をもとに回答プロンプトを実装

In [29]:
# 引数
url = "https://toukei-lab.com/achademy/?page_id=1619"
chunk_size = 400
overlap = 50


# 実際に作成した関数にて挙動の確認
article_text = scrape_article(url)
documents = chunk_text(article_text, chunk_size, overlap) #chunksの中にurlの文字列を格納

# chunks(text_chunks)とquestionに分けていく作業
# chunksのchunkを一つ一つ取り出し、ベクトル化処理に入れ込む
vectors = [vectorize_text(doc) for doc in documents]

question = "オーダーメイドプランの価格はいくら？"
question_vector = vectorize_text(question)

# documentsの中から最も高い類似度のもの(答え)を格納
similar_document = find_most_similar(question_vector, vectors,documents)

# 質問への回答を出力するプロンプトにて
answer = ask_question(question, similar_document)
print(answer)

<!DOCTYPE html>

<html data-loaded="false" data-scrolled="false" data-spmenu="closed" lang="ja">
<head>
<meta charset="utf-8"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, viewport-fit=cover" name="viewport"/>
<meta content="max-image-preview:large" name="robots">
<script>
MathJax = {
  tex: {
    inlineMath: [['$','$'],['\\(','\\)']], 
    processEscapes: true
  },
  options: {
    ignoreHtmlClass: 'tex2jax_ignore|editor-rich-text'
  }
};

</script>
<link href="//cdn.jsdelivr.net" rel="dns-prefetch"/>
<link href="https://toukei-lab.com/achademy/?feed=rss2" rel="alternate" title="スタアカ » フィード" type="application/rss+xml"/>
<link href="https://toukei-lab.com/achademy/?feed=comments-rss2" rel="alternate" title="スタアカ » コメントフィード" type="application/rss+xml"/>
<!-- SEO SIMPLE PACK 3.2.1 -->
<title>プラン一覧 | スタアカ</title>
<meta content="noindex" name="robots"/>
<meta content="プラン一覧 ライトプラン ライトプラン128